In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge


In [2]:
df = pd.read_csv('raw.csv')
df.head(2)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88


In [3]:
X = df.drop(columns  = ['math_score'])
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [4]:
y = df['math_score']
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [5]:
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import ColumnTransformer

num_features = X.select_dtypes(exclude = 'object').columns
cat_features = X.select_dtypes(include = 'object').columns

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder' , oh_transformer , cat_features),
        ('StandardScaler' , numeric_transformer , num_features)
    ]
)

In [6]:
X = preprocessor.fit_transform(X)

In [7]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [8]:
X.shape

(1000, 19)

In [9]:
# seperating the data into train and test
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size  = 0.2 , random_state=42)


In [10]:
def evaluate_model(true , predicted):
    mae = mean_absolute_error(true , predicted)
    mse = mean_squared_error(true , predicted)
    rmse = np.sqrt(mean_squared_error(true , predicted))
    r2_scores = r2_score(true , predicted)
    return mae , rmse , r2_scores
                    

In [12]:
models = {
    "linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    'Ridge':Ridge(),
    "KNeighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor(),
    "XGBRegressor":XGBRegressor(),
    "CatBoost Regressor" : CatBoostRegressor(verbose = False),
    "AdaBoost Regressor":AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train , y_train)
    
    # make Prediction
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse , model_train_r2 = evaluate_model(y_train ,y_train_pred)
    
    model_test_mae , model_test_rmse , model_test_r2 = evaluate_model(y_test  , y_test_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model Performance for Training Set')
    print("-Root Mean Square Error: {:.4f}".format(model_train_rmse))
    print("-Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    
    print('-----------------------------')
    
    print('Model Performance for test Set')
    print("-Root Mean Square Error: {:.4f}".format(model_test_rmse))
    print("-Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')




linear Regression
Model Performance for Training Set
-Root Mean Square Error: 5.3231
-Mean Absolute Error: 4.2667
- R2 Score: 0.8743
-----------------------------
Model Performance for test Set
-Root Mean Square Error: 5.3940
-Mean Absolute Error: 4.2148
- R2 Score: 0.8804


Lasso
Model Performance for Training Set
-Root Mean Square Error: 6.5938
-Mean Absolute Error: 5.2063
- R2 Score: 0.8071
-----------------------------
Model Performance for test Set
-Root Mean Square Error: 6.5197
-Mean Absolute Error: 5.1579
- R2 Score: 0.8253


Ridge
Model Performance for Training Set
-Root Mean Square Error: 5.3233
-Mean Absolute Error: 4.2650
- R2 Score: 0.8743
-----------------------------
Model Performance for test Set
-Root Mean Square Error: 5.3904
-Mean Absolute Error: 4.2111
- R2 Score: 0.8806


KNeighbors Regressor
Model Performance for Training Set
-Root Mean Square Error: 5.7077
-Mean Absolute Error: 4.5167
- R2 Score: 0.8555
-----------------------------
Model Performance for test Set

In [13]:
pd.DataFrame(list(zip(model_list , r2_list)) , columns = ['Model_name' , 'R2_score']).sort_values(by = ['R2_score'] , ascending=False)

,Model_name,R2_score
2,Ridge,0.880593
0,linear Regression,0.880433
8,AdaBoost Regressor,0.853334
5,Random Forest Regressor,0.852354
7,CatBoost Regressor,0.851632
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,KNeighbors Regressor,0.783813
4,Decision Tree,0.731423
